<a href="https://colab.research.google.com/github/kunwarsaaim/Image_Captioning/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import BatchNormalization
from keras.layers import Dense


In [23]:
def load_photos(directory):      #directory:in which photos are stored
  features = dict()
  in_layer = Input(shape=(224,224,3))
  model = VGG16(weights='imagenet',input_tensor=in_layer)
  for layer in model.layers:
    layer.trainable = False
  embedding_size = 300
  dense_input = BatchNormalization(axis=-1)(model.layers[-2].output)
  image_dense = Dense(units=embedding_size)(dense_input)
  image_model = Model(inputs=model.input,outputs=image_dense)
  print(image_model.summary())
  for file in listdir(directory):
    filename = directort + '/' + file
    #load image from file
    image = load_img(filename,target_size=(224,224))
    #convert to numpy array
    image = img_to_array(image)
    #reshape data for the model
    image = np.expand_dims(image,axis=0)
    image = preprocess_input(image)
    feature = image_model.predict(image,verbose=0)
    #get image id
    image_id = file.split(".")[0]
    #mapping image_id to numpy array converted image
    features[image_id] = feature
  return features
load_photos("directory")
                          

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

FileNotFoundError: [Errno 2] No such file or directory: 'directory'

In [27]:
def load_doc(filename):
  #open file
  file = open(filename,'r')
  text = file.read()
  file.close()
  return text


#discription for images
def load_discription(doc):
  mapping = dict()
  for line in doc.split("\n"):
    #split line by white spaces
    tokens = line.split()
    if len(line)<2:
      continue
    #first token as image id and rest as caption
    image_id,caption = tokens[0],tokens[1:]
    #remove filename from image
    image_id = image_id.split('.')[0]
    #tokens to string
    caption = " ".join(caption)
    #store first caption
    if image_id not in mapping:
      mapping[image_id] = caption
  return mapping

    

In [45]:
def text_prepross(filename):
    #load text from file
    raw_text = load_doc(filename)
    #seprate caption and image name
    text_dict = load_discription(raw_text)
    #convert dict to dataframe
    text_df = pd.DataFrame(list(text_dict.items()))
    #convert text to integers
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text_df[1])
    print(len(tokenizer.word_index)+1)
    seq = tokenizer.texts_to_sequences(text_df[1])
    #make all the tokenized caption of equal length
    maxlen = 40
    padded_seq = sequence.pad_sequences(seq,maxlen=maxlen)
    #make 2d array to list to assign to dataframe col
    padded_seq = padded_seq.tolist()
    text_df[2] = padded_seq
    return text_df
    
    

In [46]:
filename = "Flickr_TextData/Flickr8k.token.txt"

In [47]:
df = text_prepross(filename)

4421


In [48]:
df.head()

,0,1,2
0,1000268201_693b08cb0e,A child in a pink dress is climbing up a set o...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1001773457_577c3a7d70,A black dog and a spotted dog are fighting,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1002674143_1b742ab4b8,A little girl covered in paint sits in front o...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1003163366_44323f5815,A man lays on a bench while his dog sits by him .,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1007129816_e794419615,A man in an orange hat starring at something .,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
